In [ ]:
import pandas as pd

df_orders = pd.read_csv("orders.csv")
df_orders

In [ ]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()  # loads PGUSER, PGPASSWORD, PGDATABASE

conn = psycopg2.connect(
    dbname=os.getenv("PGDATABASE"),
    user=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    host="localhost",
    port="5432"
)

import pandas as pd
query = "SELECT * FROM restaurants;"

df_restaurents = pd.read_sql(query, conn)

print(df_restaurents.head())

conn.close()

df_restaurents


In [ ]:
import pandas as pd

df_users = pd.read_json("users.json")

df_users

In [ ]:
df_merged = pd.merge(df_orders, df_users,on="user_id", how="left")
df_merged

In [ ]:
df = pd.merge(df_merged, df_restaurents, on="restaurant_id", how="left")
df

In [34]:
df.to_csv("final_food_delivery_dataset.csv")

In [ ]:
#cleaning the dataset
df["order_date"] = pd.to_datetime(df["order_date"], format="%d-%m-%Y")

df


In [ ]:
orders_over_time = (
    df
    .groupby(df["order_date"].dt.to_period("M"))
    .size()
    .reset_index(name="order_count")
)
orders_over_time

In [ ]:
revenue_trend = (
    df
    .groupby(df["order_date"].dt.to_period("M"))["total_amount"]
    .sum()
    .reset_index()
)

revenue_trend

df["quarter"] = df["order_date"].dt.quarter
quarter_revenue = df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
quarter_revenue
top_quarter = quarter_revenue.idxmax()
top_revenue = quarter_revenue.max()

print(f"Quarter with highest revenue: Q{top_quarter}")
print(f"Total revenue: ₹{top_revenue:.2f}")



In [ ]:
#Which city has the highest total revenue (total_amount) from Gold members?  
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


In [ ]:
#Which cuisine has the highest average order value across all orders?
cuisine_aov = (
    df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_aov


In [ ]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_total_spend = (
    df
    .groupby("user_id")["total_amount"]
    .sum()
)


high_value_users = user_total_spend[user_total_spend > 1000]


count_users = high_value_users.count()

count_users

In [ ]:
#Which restaurant rating range generated the highest total revenue?

rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = [
    "3.0–3.5",
    "3.6–4.0",
    "4.1–4.5",
    "4.6–5.0"
]

df["rating_range"] = pd.cut(
    df["rating"],
    bins=rating_bins,
    labels=rating_labels,
    right=True,
    include_lowest=True
)
rating_revenue = (
    df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


In [ ]:
#Among Gold members, which city has the highest average order value?
gold_city_aov = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_aov

In [ ]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

restaurants_per_cuisine = (
    df.groupby("cuisine")["restaurant_id"]
      .nunique()
      .reset_index(name="restaurant_count")
)

revenue_per_cuisine = (
    df.groupby("cuisine")["total_amount"]
      .sum()
      .reset_index(name="total_revenue")
)

cuisine_analysis = restaurants_per_cuisine.merge(
    revenue_per_cuisine,
    on="cuisine"
)
cuisine_analysis.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
)

cuisine_analysis


In [ ]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
total_orders = df["order_id"].nunique()
gold_orders = df[df["membership"] == "Gold"]["order_id"].nunique()
percentage_gold_orders = round((gold_orders / total_orders) * 100)
percentage_gold_orders



In [ ]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]
df_options = df[df["restaurant_name_x"].isin(options)]
stats = (
    df_options
    .groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
    .reset_index()
)
stats_filtered = stats[stats["total_orders"] < 20]

answer = stats_filtered.sort_values(
    by="avg_order_value",
    ascending=False
).head(1)

answer


In [ ]:
top_combo = (
    df[
        ((df["membership"] == "Gold") & (df["cuisine"].isin(["Indian", "Italian"]))) |
        ((df["membership"] == "Regular") & (df["cuisine"].isin(["Indian", "Chinese"])))
    ]
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .idxmax()
)

top_revenue = (
    df[
        ((df["membership"] == "Gold") & (df["cuisine"].isin(["Indian", "Italian"]))) |
        ((df["membership"] == "Regular") & (df["cuisine"].isin(["Indian", "Chinese"])))
    ]
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .max()
)

print(f"Highest revenue combination: {top_combo[0]} + {top_combo[1]}")
print(f"Revenue: ₹{top_revenue:.2f}")

In [ ]:
#How many total orders were placed by users with Gold membership?
# Filter Gold members and count orders
gold_orders_count = df[df["membership"] == "Gold"]["order_id"].count()

print(f"Total orders placed by Gold members: {gold_orders_count}")


In [ ]:
hyderabad_revenue = round(df[df["city"] == "Hyderabad"]["total_amount"].sum())

hyderabad_revenue

In [ ]:
distinct_users = df["user_id"].nunique()
distinct_users

In [ ]:
gold_aov = round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)
gold_aov

In [ ]:
high_rating_orders = df[df["rating"] >= 4.5]["order_id"].count()
high_rating_orders

In [ ]:

gold_df = df[df["membership"] == "Gold"]


top_city = gold_df.groupby("city")["total_amount"].sum().idxmax()


orders_in_top_city = gold_df[gold_df["city"] == top_city]["order_id"].count()

top_city


In [ ]:
total_rows = df.shape[0]
total_rows
